In [1]:
!apt-get -q -y install swig
!git clone https://www.github.com/ildoonet/tf-openpose
!pwd
%cd tf-openpose
!pip3 install -r requirements.txt
%cd models/graph/cmu
!bash download.sh
%cd ../../../
%cd tf_pose/pafprocess
!ls
!swig -python -c++ pafprocess.i && python3 setup.py build_ext --inplace
%cd ../../

from google.colab import drive
drive.mount('./gdrive')

Reading package lists...
Building dependency tree...
Reading state information...
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  swig3.0
Suggested packages:
  swig-doc swig-examples swig3.0-examples swig3.0-doc
The following NEW packages will be installed:
  swig swig3.0
0 upgraded, 2 newly installed, 0 to remove and 7 not upgraded.
Need to get 1,100 kB of archives.
After this operation, 5,822 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 swig3.0 amd64 3.0.12-1 [1,094 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 swig amd64 3.0.12-1 [6,460 B]
Fetched 1,100 kB in 2s (455 kB/s)
Selecting previously unselected package swig3.0.
(Reading database ... 130824 files and directories currently installed.)
Preparing to unpack .../swig3.0_3.0.12-1_amd64.deb ...
Unpacking swig3.0 (3.

/content/tf-openpose/models/graph/cmu
[download] model graph : cmu
--2019-05-20 09:03:51--  http://download686.mediafire.com/att1x9ym8ieg/qlzzr20mpocnpa3/graph_opt.pb
Resolving download686.mediafire.com (download686.mediafire.com)... 205.196.120.134
Connecting to download686.mediafire.com (download686.mediafire.com)|205.196.120.134|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 209299198 (200M) [application/octet-stream]
Saving to: ‘/content/tf-openpose/models/graph/cmu/graph_opt.pb’

/content/tf-openpos 100%[===================>] 199.60M   983KB/s    in 3m 19s  

2019-05-20 09:07:11 (1.00 MB/s) - ‘/content/tf-openpose/models/graph/cmu/graph_opt.pb’ saved [209299198/209299198]

[download] end
/content/tf-openpose
/content/tf-openpose/tf_pose/pafprocess
__init__.py  pafprocess.cpp  pafprocess.i   pafprocess_wrap.cpp  setup.py
numpy.i      pafprocess.h    pafprocess.py  README.md
running build_ext
building '_pafprocess' extension
swigging pafprocess.i to pafproc

### **これは自分が入力する部分** *

In [0]:
#元の動画の情報を見て、1920 ×　1080　のようにサイズを合わせる
w = 1024
h = 576

#動画の半分のスピードになります。　　３０にすれば1倍の速さ
speed = 10

#movie_nameにGoogle Driveの動画名を入れる
movie_name = 'tennis_serve.MOV'
!cp './gdrive/My Drive/tennis_serve.MOV' "./"


In [30]:
#棒人間
import numpy as np
import argparse, cv2, os
from tf_pose import common
from tf_pose.estimator import TfPoseEstimator
from tf_pose.networks import get_graph_path, model_wh
from PIL import Image

img_outdir = './img３'
os.makedirs(img_outdir, exist_ok=True)

# 動画作成
fourcc = cv2.VideoWriter_fourcc(*'XVID')
video  = cv2.VideoWriter('output_line_output.mp4', fourcc, speed, (w, h))


if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='tf-pose-estimation Video')

    outimg_files = []
    count = 0
    
    e = TfPoseEstimator(get_graph_path('mobilenet_thin'), target_size=(w, h))
    cap = cv2.VideoCapture(movie_name)

    # 動画用の画像作成
    while True:
        ret, image = cap.read()

        if ret == True:
            # １フレームずつ処理
            count += 1
            img = np.zeros((h, w, 3), dtype=np.uint8)
            humans = e.inference(image, resize_to_default=(w > 0 and h > 0), upsample_size=4)
            image = e.draw_humans(img, humans, imgcopy=False)

            # 画像出力
            outimg_file = '{}/{:05d}.jpg'.format(img_outdir, count)
            cv2.imwrite(outimg_file, image)
            video.write(image) 

        else:
            break
    video.release()

[2019-05-20 10:04:44,703] [TfPoseEstimator] [INFO] loading graph from /content/tf-openpose/models/graph/mobilenet_thin/graph_opt.pb(default size=1024x576)
2019-05-20 10:04:44,703 INFO loading graph from /content/tf-openpose/models/graph/mobilenet_thin/graph_opt.pb(default size=1024x576)


In [20]:
#背景を残しつつ姿勢推定
# from PIL import Image

# # 動画作成
fourcc = cv2.VideoWriter_fourcc(*'XVID')
video  = cv2.VideoWriter('output.mp4', fourcc, speed, (w, h))

if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='tf-pose-estimation Video')

    outimg_files = []
    count = 0
    e = TfPoseEstimator(get_graph_path('mobilenet_thin'), target_size=(w, h))
    cap = cv2.VideoCapture(movie_name)

    # 動画用の画像作成
    while True:
        ret, image = cap.read()

        if ret == True:
            # １フレームずつ処理
            count += 1
            humans = e.inference(image, resize_to_default=(w > 0 and h > 0), upsample_size=4)
            image = TfPoseEstimator.draw_humans(image, humans, imgcopy=False)

            # 画像出力
            outimg_file = '{}/{:05d}.jpg'.format(img_outdir, count)
            cv2.imwrite(outimg_file, image)
            video.write(image) 

        else:
            break
    video.release()

[2019-05-20 09:50:30,828] [TfPoseEstimator] [INFO] loading graph from /content/tf-openpose/models/graph/mobilenet_thin/graph_opt.pb(default size=1280x720)
2019-05-20 09:50:30,828 INFO loading graph from /content/tf-openpose/models/graph/mobilenet_thin/graph_opt.pb(default size=1280x720)


In [0]:
!cp './output_line_K_M2.mp4'  './gdrive/My Drive/'
!cp './output_K_M2.mp4'  './gdrive/My Drive/'

In [14]:
import argparse
import logging
import time
import os

import cv2
import numpy as np
import matplotlib.pyplot as plt

from tf_pose import common
from tf_pose.estimator import TfPoseEstimator
from tf_pose.networks import get_graph_path, model_wh


img_outdir = './img'
os.makedirs(img_outdir, exist_ok=True)

# 動画作成
fourcc = cv2.VideoWriter_fourcc('m','p','4', 'v')
video  = cv2.VideoWriter('output_tennis_serve.mp4', fourcc, 30.0, (1920, 1080))


if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='tf-pose-estimation Video')

    outimg_files = []
    count = 0
    w = 1920
    h = 1080
    e = TfPoseEstimator(get_graph_path('mobilenet_thin'), target_size=(w, h))

    
    cap = cv2.VideoCapture('tennis_serve.MOV')

    # 動画用の画像作成
    while True:
        ret, image = cap.read()

        if ret == True:
            # １フレームずつ処理
            count += 1
            if count % 100 == 0:
                print('Image No.{0}'.format(count))

            humans = e.inference(image, resize_to_default=(w > 0 and h > 0), upsample_size=4)
            image = TfPoseEstimator.draw_humans(image, humans, imgcopy=False)

            # 画像出力
            outimg_file = '{}/{:05d}.jpg'.format(img_outdir, count)
            cv2.imwrite(outimg_file, image)
            video.write(image)       

        else:
            break
    video.release()

[2019-05-20 09:35:23,788] [TfPoseEstimator] [INFO] loading graph from /content/tf-openpose/models/graph/mobilenet_thin/graph_opt.pb(default size=1920x1080)
2019-05-20 09:35:23,788 INFO loading graph from /content/tf-openpose/models/graph/mobilenet_thin/graph_opt.pb(default size=1920x1080)


Image No.：100
